In [2]:

%load_ext autoreload
%autoreload 2


In [3]:
import os
os.chdir('../')

import sys 
sys.path.append('../')
import time
import pandas as pd 
import numpy as np
import torch
import openai
from tqdm import tqdm
from dotenv import load_dotenv
from trainer.transformer_utilts import *
from helper.dataloader import * 
import pickle
from peft import get_peft_model
from model.MF import sentenceT5Classification
import argparse
from tqdm import tqdm
from peft import LoraConfig, TaskType
import logging
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer


logging.basicConfig(filename=f'./logs/{time.time()}.log',level=logging.INFO)
logging.info(f'Experiment started at {time.time()}')


def parse_args(notebook = False):  # Parse command line arguments
    parser = argparse.ArgumentParser(description="LLM4RecSys")
    parser.add_argument("--data_name", default='ml-1m', type=str)
    parser.add_argument("--direction", default='down', type=str)
    parser.add_argument("--bs", default=32, type=int)
    parser.add_argument("--target_index", default=49, type=int)
    parser.add_argument("--num_samples", default=250, type=int)
    parser.add_argument("--max_l", default=260, type=int)
    parser.add_argument("--debug_prompts", default=False, type=bool)
    parser.add_argument("--metadata_path",default='./data_preprocessed/books/train.csv',type=str)
    args = parser.parse_args() if not notebook else parser.parse_args(args=[])
    args.scratch = '/home/mila/e/emiliano.penaloza/scratch'
    return args
args = parse_args(True)
args.data_name = 'books'


load_dotenv()
max_l = args.max_l
key = os.getenv("OPEN-AI-SECRET")
openai.api_key = key
tokenizer = AutoTokenizer.from_pretrained("t5-large")
prompts,rec_dataloader,num_movies,val_dataloader,test_dataloader,_,_= load_data(args,tokenizer,0,1)

# Load model and mapping dicts
path = f'{args.scratch}/saved_model/{args.data_name}/t5_classification_embedding_module_t5_classification_l2_lambda_0.0_lora_r_32_scheduler_cosine_warmup_0.001_0.2.csv.pt'
model = sentenceT5Classification.from_pretrained('t5-large', num_labels=num_movies)

lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, r=32, lora_alpha=16, lora_dropout=0,
                            target_modules=["q", "v"],
                            modules_to_save=['classification_head'])
model = get_peft_model(model, lora_config)
model.load_state_dict(torch.load(path ,map_location=torch.device('cpu')))
model.to(0)


/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model

data_path='./data_preprocessed/books/'
Max Prompt Length 260
Number of Users is num_users=3534
Number of Movies is num_movies=14871


Some weights of sentenceT5Classification were not initialized from the model checkpoint at t5-large and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): sentenceT5Classification(
      (transformer): T5EncoderModel(
        (shared): Embedding(32128, 1024)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 1024)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): lora.Linear(
                      (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                      (lora_dropout): ModuleDict(
                        (default): Identity()
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=1024, out_features=32, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=32, out_features=1024, bias=False)
                    

In [21]:

with open(f'./data_preprocessed/{args.data_name}/show2id.pkl','rb') as f:
    movie_id_map = pickle.load(f)
    movie_id_map = {v:k for k,v in movie_id_map.items()}
    


    

item_id_to_title = map_id_to_title( args.metadata_path,data = 'books')
item_id_to_genre = map_id_to_genre(args.metadata_path,data = 'books')
item_title_to_genre = {item_title: item_id_to_genre[item_id] for item_id, item_title in item_id_to_title.items()}



def tokenize_prompt(tokenizer,prompt,max_l):
    encodings = tokenizer([prompt],padding=True, truncation=True,max_length=max_l,
                          return_tensors='pt')
    return encodings


def genrewise_ndcg(genre_movies, genre, min_k=0, max_k=None):
    relevance = [1 if genre in g   else 0 for g in genre_movies[min_k:max_k]]
    dcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(relevance))
    idcg = sum(1 / np.log2(i + 2) for i in range(len(relevance)))
    ndcg = dcg / idcg if idcg > 0 else 0
    
    return ndcg

# gets diffs in recommendations between two summaries
def get_difs(s1,s2,device = 0,genre = 'Action' ,max_k = 20 ):

    s_1_encodings = tokenize_prompt(tokenizer,s1,max_l)
    s_2_encodings = tokenize_prompt(tokenizer,s2,max_l)
    for k in s_1_encodings:
        s_1_encodings[k] = s_1_encodings[k].to(device)
    for k in s_2_encodings:
        s_2_encodings[k] = s_2_encodings[k].to(device)
    out_s1 = model(**s_1_encodings)
    out_s2 = model(**s_2_encodings)
    l = torch.topk(torch.softmax(out_s1.logits,dim = 1),1000)
    indices_1 = l.indices[0]
    movie_titles_1 = [item_id_to_title[item_id_map[index_1.item()]] for index_1 in indices_1] if args.data_name == 'ml-1m' else [item_id_to_title[index_1.item()] for index_1 in indices_1]
    l2 = torch.topk(torch.softmax(out_s2.logits,dim = 1),1000)
    indices_2 = l2.indices[0]
    movie_titles_2 = [item_id_to_title[item_id_map[index_2.item()]] for index_2 in indices_2] if args.data_name == 'ml-1m' else [item_id_to_title[index_2.item()] for index_2 in indices_2]
    genre_movies_1 = [item_title_to_genre[movie_title] for movie_title in movie_titles_1]
    genre_movies_2 = [item_title_to_genre[movie_title] for movie_title in movie_titles_2]
    ndcg_1 = genrewise_ndcg(genre_movies_1,genre,min_k = 0,max_k = max_k)
    ndcg_2 = genrewise_ndcg(genre_movies_2,genre,min_k = 0,max_k = max_k)
    delta = ndcg_1 - ndcg_2    

    rankings1 = [(movie_title,genre) for movie_title,genre in zip(movie_titles_1,genre_movies_1)]
    rankings2 = [(movie_title,genre) for movie_title,genre in zip(movie_titles_2,genre_movies_2)]



    return delta,rankings1,rankings2


In [22]:
counts = Counter(sum([v.split("|") for v in item_id_to_genre.values()],[]))
#keep counts if above 200 
counts = {k:v for k,v in counts.items() if v > 100}
genre_set = list(counts.keys())
genre_set = ', '.join(genre_set)
k_list= list(prompts.keys())
k_sample = random.sample(k_list,500)

In [23]:

np.random.seed(2024)
# Prompt GPT to change summaries 
model.eval()
moved_up = []
moved_down = []
average_medians_down = []
average_medians_up = []
move_up_genres =[]
move_down_genres = []
outputs = []
keys = []
deltas_ndcg_up = []
deltas_ndcg_down = []
deltas_ndcg_up = []
deltas_ndcg_down = []
for k in (pbar:=tqdm(k_sample)): 
    s = prompts[k]
        
    prompt = \
                f"""
                You are a professional editor please identify the users preferred genres from the following:
                {genre_set}
                """
    user_prompt =\
                f"""
                Please identify the users most favorite genre from the following summary and the least favorite genre: 
                in the format Favorite: [genre]\n Least Favorite: [genre]
                {s}.
                """
                
    msg = [
                {
                    "role": "system",
                    "content": prompt

                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]

    genres = openai.ChatCompletion.create(
                    model='gpt-4-1106-preview',
                    messages=msg,
                    max_tokens=300,
                    temperature=0.001,
                    seed=2024,
                )['choices'][0]['message']['content']
    lines = genres.split('\n')
    favorite_genre = lines[0].split(': ')[1]
    logging.info(f"{favorite_genre=}")
    least_favorite_genre = lines[1].split(': ')[1]
    logging.info(f"{least_favorite_genre=}")
    
    msg = [
                {
                    "role": "system",
                    "content": prompt

                },
                {
                    "role": "user",
                    "content": user_prompt
                },
                {'role': 'assistant',
                'content': genres},
                {'role':'user',
                'content':
                f'Now using this setup write the a new summary in the same style that reflects that {favorite_genre} is your least favorite\
                    and {least_favorite_genre} is your favorite only output the full summary keep the format and length the same' }
            ]

    gpt_output = openai.ChatCompletion.create(
                    model='gpt-4-1106-preview',
                    messages=msg,
                    max_tokens=300,
                    temperature=0.0000001,
                    seed=2024,
                )['choices'][0]['message']['content']


    delta_down,rankings1_down,rankings2_down = get_difs(s,gpt_output,genre = favorite_genre)
    delta_up,rankings1_up,rankings2_up = get_difs(s,gpt_output,genre = least_favorite_genre)
    move_down_genres.append(favorite_genre)
    move_up_genres.append(least_favorite_genre)
    outputs.append(gpt_output)  
    keys.append(k)
    deltas_ndcg_down.append(delta_down)
    deltas_ndcg_up.append(delta_up)

    
    pbar.set_description(f"average ndcgs up {np.mean(deltas_ndcg_up)} average ndcg down {np.mean(deltas_ndcg_down)}")    
    logging.info(f"average ndcgs up {np.mean(deltas_ndcg_up)} average ndcg down {np.mean(deltas_ndcg_down)}")
    

data = {
    'move_down_genres': move_down_genres,
    'move_up_genres': move_up_genres,
    'outputs': outputs,
    'keys': keys,
    'deltas_ndcg_down': deltas_ndcg_down,
    'deltas_ndcg_up': deltas_ndcg_up,
}


df = pd.DataFrame(data)
df.to_csv(f'./results/{args.data_name}/gpt4_results_large_{args.direction}.csv')
logging.info(f'./results/{args.data_name}/gpt4_results_large_{args.direction}.csv')



average ndcgs up -0.263729111333051 average ndcg down 0.22492106360262382: 100%|██████████| 500/500 [1:41:50<00:00, 12.22s/it]  
